In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

In [2]:
# LIEN ACTEURS

df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [3]:
df_name_basics['knownForTitles'] = df_name_basics['knownForTitles'].apply(lambda x : x.split(','))
df_name_basics_exp = df_name_basics.explode('knownForTitles')

In [4]:
df_name_basics = 0

In [5]:
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

In [6]:
df_title_principals = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self') | (df_title_principals['category'] == 'director') | (df_title_principals['category'] == 'writer')]

In [7]:
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

In [8]:
df_title_crew['directors'] = df_title_crew['directors'].apply(lambda x : x.split(','))
df_title_crew['writers'] = df_title_crew['writers'].apply(lambda x : x.split(','))
df_title_crew_exp = df_title_crew.explode('directors')
df_title_crew_exp = df_title_crew.explode('writers')
df_title_crew_exp['directors'] = df_title_crew_exp['directors'].apply(lambda x : ''.join(x))
df_title_crew_exp['writers'] = df_title_crew_exp['writers'].apply(lambda x : ''.join(x))

In [9]:
df_title_crew = 0

In [10]:
df_title_principals_name = pd.merge(df_title_principals,
                                    df_name_basics_exp,
                                    on = 'nconst',
                                    how = 'left')

df_title_principals = 0

In [19]:
df_title_crew_name = pd.merge(df_title_crew_exp,
                                    df_name_basics_exp,
                                    right_on = 'nconst',
                                    left_on = 'directors',
                                    how = 'left')

df_title_crew_exp = 0

In [21]:
df_title_crew_name = df_title_crew_name.rename(columns = {'primaryName' : 'director_primaryName'})

In [24]:
df_title_crew_name2 = pd.merge(df_title_crew_name,
                                    df_name_basics_exp,
                                    right_on = 'nconst',
                                    left_on = 'writers',
                                    how = 'left')

In [25]:
df_title_crew_name = 0

In [29]:
df_title_crew_name2 = df_title_crew_name2.rename(columns = {'primaryName' : 'writers_primaryName'})